# Data treatment

**Objective:** read a dataset filtered by studied year(2021), treat such as necessary (removing rows, renaming, dropping columns...), and generate a new file called "fluxo_escolar_2021_cleaned.xlsx" in the "output" folder.

## Importing packages

In [27]:
import numpy as np
import pandas as pd
import datetime
import unicodedata

## Importing dataset

In [82]:
df = pd.read_excel('./data/Fluxo_Escolar_2021.xlsx',engine='openpyxl')
df.head()

,ANO_LETIVO,NM_DIRETORIA,NM_MUNICIPIO,CD_REDE_ENSINO,CD_ESCOLA,NM_COMPLETO_ESCOLA,CD_TP_IDENTIFICADOR,APR_1,REP_1,ABA_1,APR_2,REP_2,ABA_2,APR_3,REP_3,ABA_3
0,2021,PIRAJU,PIRAJU,1,34320,JOAQUIM GUILHERME MOREIRA PORTO DOUTOR,8,0.0,0.0,0.0,100.00,0.00,0.0,100.0,0.0,0.0
1,2021,PIRAJU,PIRAJU,1,34344,QUINZINHO CAMARGO PREFEITO,8,0.0,0.0,0.0,99.56,0.44,0.0,94.0,2.0,4.0
2,2021,PIRAJU,PIRAJU,1,34356,NHONHO BRAGA CORONEL,8,0.0,0.0,0.0,100.00,0.00,0.0,100.0,0.0,0.0
3,2021,PIRAJU,PIRAJU,1,34459,ATALIBA LEONEL,8,0.0,0.0,0.0,100.00,0.00,0.0,100.0,0.0,0.0
4,2021,AMERICANA,AMERICANA,1,17152,MAURA ARRUDA GUIDOLIN PROFA,8,100.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0


## Dropping unused columns

In [41]:
drop_list = ['ANO_LETIVO',
             'NM_DIRETORIA',
             'NM_COMPLETO_ESCOLA',
             'CD_TP_IDENTIFICADOR',
             'APR_1',
             'REP_1',
             'ABA_1',
             'APR_2',
             'REP_2',
             'ABA_2']

df_column_dropped = df.drop(drop_list, axis='columns')
df_column_dropped.head()

,NM_MUNICIPIO,CD_REDE_ENSINO,CD_ESCOLA,APR_3,REP_3,ABA_3
0,PIRAJU,1,34320,100.0,0.0,0.0
1,PIRAJU,1,34344,94.0,2.0,4.0
2,PIRAJU,1,34356,100.0,0.0,0.0
3,PIRAJU,1,34459,100.0,0.0,0.0
4,AMERICANA,1,17152,0.0,0.0,0.0


## Renaming columns

In [48]:
rename_list = {'ANO_LETIVO': 'ano',
               'NM_MUNICIPIO': 'municipio',
               'CD_REDE_ENSINO': 'id_rede_ensino',
               'CD_ESCOLA': 'id_escola',
               'APR_3': 'aprovado',
               'REP_3': 'reprovado',
               'ABA_3': 'abandono'}

df_column_renamed = df_column_dropped.rename(columns=rename_list)
df_column_renamed.head()

,municipio,id_rede_ensino,id_escola,aprovado,reprovado,abandono
0,PIRAJU,1,34320,100.0,0.0,0.0
1,PIRAJU,1,34344,94.0,2.0,4.0
2,PIRAJU,1,34356,100.0,0.0,0.0
3,PIRAJU,1,34459,100.0,0.0,0.0
4,AMERICANA,1,17152,0.0,0.0,0.0


## Filtering

In [59]:
print('Dataframe length before filtering:', len(df_column_rename))

Dataframe length before filtering: 5194


### Removing rows that contains hyphen

In [72]:
df_cleaned = df_column_renamed[df_column_renamed != "-"]
df_cleaned = df_cleaned[df_cleaned != datetime.time(0, 0)]
df_cleaned = df_cleaned.dropna()

print('Dataframe length: ', len(df_cleaned))

Dataframe length:  5194


### Removing rows that contains invalid datetime format

In [73]:
df_cleaned = df_cleaned[df_cleaned != datetime.time(0, 0)]

print('Dataframe length: ', len(df_cleaned))

Dataframe length:  5194


### Removing rows that contains invalid values

In [74]:
df_cleaned = df_cleaned.loc[(df_cleaned['aprovado'] > 0) | (df_cleaned['reprovado'] > 0)]

print('Dataframe length: ', len(df_cleaned))

Dataframe length:  3745


## County name corrections

In [75]:
df_cleaned_unique_county_list = df_cleaned['municipio'].unique().tolist()

ibge_counties_data = pd.read_excel('data/ibge_sp_counties_data.xlsx')
sp_counties_list = ibge_counties_data.loc[:, 'Município'].tolist()

def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

sp_counties_normalized_list = [ strip_accents(i).replace(';', '').upper() for i in sp_counties_list]

diff1 = set(sp_counties_normalized_list).difference(set(df_cleaned_unique_county_list))
diff2 = set(df_cleaned_unique_county_list).difference(set(sp_counties_normalized_list))

print(f'''Comparing results:

sp_counties_list values that do not in df_cleaned_unique_county_list: {diff1 or None}
df_cleaned_unique_county_list values that do not in sp_counties_list: {diff2 or None}''')

Comparing results:

sp_counties_list values that do not in df_cleaned_unique_county_list: {'SANTANA DE PARNAIBA', 'BIRITIBA MIRIM', 'FLORINEA'}
df_cleaned_unique_county_list values that do not in sp_counties_list: {'FLORÍNEA', 'BIRITIBA-MIRIM'}


**Note:**
'SANTANA DE PARNAIBA' is a county that is not in case study dataframe, so will be ignored

In [78]:
df_cleaned['municipio'] = df_cleaned['municipio'].replace('BIRITIBA-MIRIM', 'BIRITIBA MIRIM')
df_cleaned['municipio'] = df_cleaned['municipio'].replace('FLORÍNEA', 'FLORINEA')
df_cleaned['municipio'] = df_cleaned['municipio'].replace('MOGI-GUACU', 'MOGI GUACU')
df_cleaned['municipio'] = df_cleaned['municipio'].replace('MOGI-MIRIM', 'MOGI MIRIM')

df_cleaned.head()

,municipio,id_rede_ensino,id_escola,aprovado,reprovado,abandono
0,PIRAJU,1,34320,100.00,0.00,0.00
1,PIRAJU,1,34344,94.00,2.00,4.00
2,PIRAJU,1,34356,100.00,0.00,0.00
3,PIRAJU,1,34459,100.00,0.00,0.00
6,AMERICANA,1,17176,79.82,19.29,0.89


## Exporting cleaned dataset

In [77]:
df_cleaned.to_excel('output/fluxo_escolar_2021_cleaned.xlsx', encoding="utf-8")

/Users/maicon_araujo/Documents/personal_projects/student-analysis/venv/lib/python3.8/site-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
